In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyarrow as pa
import pyarrow.parquet as pq
import geopandas as gpd
import seaborn as sns

#Packages
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import numpy as np
from scipy import stats
pd.set_option('display.max_columns', None)

from scipy.stats import skew, kurtosis

import pygris

In [ ]:
# rm(list=ls()) 

# library(devtools)


# library(choroplethrZip)
# library(tidyverse)

# https://github.com/arilamstein/choroplethrZip

In [2]:
df = pd.read_parquet("C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/FimaNfipClaims.parquet.gzip")

In [3]:
df.head()

,agricultureStructureIndicator,asOfDate,basementEnclosureCrawlspaceType,policyCount,crsClassificationCode,dateOfLoss,elevatedBuildingIndicator,elevationCertificateIndicator,elevationDifference,baseFloodElevation,ratedFloodZone,houseWorship,locationOfContents,lowestAdjacentGrade,lowestFloorElevation,numberOfFloorsInTheInsuredBuilding,nonProfitIndicator,obstructionType,occupancyType,originalConstructionDate,originalNBDate,amountPaidOnBuildingClaim,amountPaidOnContentsClaim,amountPaidOnIncreasedCostOfComplianceClaim,postFIRMConstructionIndicator,rateMethod,smallBusinessIndicatorBuilding,totalBuildingInsuranceCoverage,totalContentsInsuranceCoverage,yearOfLoss,primaryResidenceIndicator,buildingDamageAmount,buildingDeductibleCode,netBuildingPaymentAmount,buildingPropertyValue,causeOfDamage,condominiumCoverageTypeCode,contentsDamageAmount,contentsDeductibleCode,netContentsPaymentAmount,contentsPropertyValue,disasterAssistanceCoverageRequired,eventDesignationNumber,ficoNumber,floodCharacteristicsIndicator,floodWaterDuration,floodproofedIndicator,floodEvent,iccCoverage,netIccPaymentAmount,nfipRatedCommunityNumber,nfipCommunityNumberCurrent,nfipCommunityName,nonPaymentReasonContents,nonPaymentReasonBuilding,numberOfUnits,buildingReplacementCost,contentsReplacementCost,replacementCostBasis,stateOwnedIndicator,waterDepth,floodZoneCurrent,buildingDescriptionCode,rentalPropertyIndicator,state,reportedCity,reportedZipCode,countyCode,censusTract,censusBlockGroupFips,latitude,longitude,id
0,0,2020-01-22T16:55:53.194Z,NaN,1,8.0,1998-02-07T00:00:00.000Z,0,nan,NaN,NaN,X,0,NaN,NaN,NaN,4.0,0,10.0,1.0,1963-01-01T00:00:00.000Z,1997-01-11T00:00:00.000Z,NaN,NaN,NaN,0,7,0,200000.0,50000.0,1998,0,382.0,0,0.00,937.0,1,N,NaN,0,0.0,NaN,0.0,None,612.0,NaN,0.0,0,Pineapple Express - Southern,15000.0,0.0,60294.0,None,"OCEANSIDE, CITY OF",97.0,1.0,1.0,937.0,NaN,A,0,0.0,None,NaN,0,CA,Currently Unavailable,92056.0,6073.0,6.073019e+09,6.073019e+10,33.2,-117.3,23dcb0d8-3e61-45bf-899f-b951946ce2ff
1,0,2020-01-22T16:55:53.194Z,NaN,1,8.0,2005-08-29T00:00:00.000Z,0,nan,NaN,NaN,X,0,NaN,NaN,NaN,2.0,0,NaN,1.0,1967-07-01T00:00:00.000Z,1990-07-12T00:00:00.000Z,NaN,NaN,NaN,0,7,0,100000.0,40000.0,2005,1,NaN,0,0.00,NaN,1,N,NaN,0,0.0,NaN,0.0,None,654.0,NaN,0.0,0,Hurricane Katrina,30000.0,0.0,225203.0,None,NEW ORLEANS/ORLEANS PARISH*,6.0,6.0,1.0,NaN,NaN,A,0,0.0,None,NaN,0,LA,Currently Unavailable,70131.0,22071.0,2.207100e+10,2.207100e+11,29.9,-90.0,55783cdd-ccbd-4b19-930b-072def248507
2,0,2020-01-22T16:55:53.194Z,NaN,1,9.0,1998-09-28T00:00:00.000Z,0,nan,NaN,NaN,X,0,NaN,NaN,NaN,1.0,0,10.0,1.0,1972-01-01T00:00:00.000Z,1997-07-24T00:00:00.000Z,8813.21,1720.0,0.0,0,1,0,100000.0,50000.0,1998,1,9313.0,0,8813.21,80000.0,1,N,2220.0,0,1720.0,0.0,0.0,None,133.0,NaN,0.0,0,Hurricane Georges (Panhandle),15000.0,0.0,120274.0,None,SANTA ROSA COUNTY *,NaN,NaN,1.0,100000.0,0.0,A,0,0.0,None,NaN,0,FL,Currently Unavailable,32566.0,12113.0,1.211301e+10,1.211301e+11,30.4,-86.9,bfb5922b-1b21-4882-b1d4-b3825ff53e37
3,0,2019-09-19T13:45:58.425Z,1.0,1,9.0,1994-10-07T00:00:00.000Z,0,nan,NaN,NaN,X,0,NaN,NaN,NaN,2.0,0,10.0,1.0,1960-01-01T00:00:00.000Z,1993-10-01T00:00:00.000Z,2906.00,0.0,0.0,0,7,0,100000.0,25000.0,1994,0,4428.0,0,2906.00,100000.0,1,N,NaN,0,0.0,NaN,0.0,None,NaN,NaN,0.0,0,None,NaN,0.0,450026.0,None,"BEAUFORT, CITY OF",97.0,NaN,1.0,0.0,NaN,A,0,0.0,None,NaN,0,SC,Currently Unavailable,29902.0,45013.0,4.501300e+10,4.501300e+11,32.4,-80.7,c1cf6e00-1e6d-4493-93fc-eb430ef15495
4,0,2019-09-19T13:45:58.425Z,NaN,1,8.0,1996-03-11T00:00:00.000Z,0,nan,NaN,NaN,X,0,NaN,NaN,NaN,1.0,0,NaN,1.0,1988-01-01T00:00:00.000Z,1996-01-11T00:00:00.000Z,3875.53,1545.0,0.0,1,7,0,100000.0,25000.0,1996,1,5252.0,0,3875.53,100000.0,1,N,3115.0,0,1545.0,0.0,0.0,None,NaN,NaN,0.0,0,None,NaN,0.0,125092.0,None,BREVARD COUNTY *,NaN,NaN,1.0,0.0,0.0,A,0,0.0,None,1.0,0,FL,Currently Unavailable,32940.0,12009.0,1.200906e+10,1.200906e+11,28.3,-80.7,cad8334c-13f2-4837-bdcf-e09591197ff7


In [4]:
df['reportedZipCode'].dtypes

dtype('float64')

In [5]:
df['reportedZipCode'].isna().sum() / df.shape[0]

0.021602852983582808

In [6]:
df['censusBlockGroupFips'].isna().sum() / df.shape[0]

0.053005097819786226

In [7]:
df['latitude'].isna().sum() / df.shape[0]

0.015378203744076599

In [8]:
df['longitude'].isna().sum() / df.shape[0]

0.015378203744076599

In [9]:
missing_all_three = df['reportedZipCode'].isna() | df['censusBlockGroupFips'].isna() | df['latitude'].isna()

# To get the number of such rows
num_missing_all_three = missing_all_three.sum()

# To get the proportion of such rows
proportion_missing_all_three = num_missing_all_three / df.shape[0]

In [10]:
proportion_missing_all_three*100

6.149075821846406

In [23]:
#Zipcode list

list_zipcode = df['reportedZipCode'].dropna().drop_duplicates().astype(int).astype(str).tolist()

# add preceding 0 to make 4-digit zip codes into 5-digit
list_zipcode = [zipcode.zfill(5) for zipcode in list_zipcode]

# temp = pd.DataFrame(list_censusBG, columns=['ZipCode'])

# temp.to_csv('C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/ZipCode.csv', index=False)

In [12]:
#CensusBG list

list_censusBG = df['censusBlockGroupFips'].dropna().drop_duplicates().tolist()

list_censusBG = [str(int(float(i))) for i in list_censusBG]

# add preceding 0 to make 4-digit zip codes into 5-digit
list_censusBG = [censusBG.zfill(12) for censusBG in list_censusBG]

# temp = pd.DataFrame(list_censusBG, columns=['censusBG'])

# temp.to_csv('C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/censusBG.csv', index=False)

In [13]:
#lat long df

df_lat_long = df[['latitude', 'longitude']].dropna().drop_duplicates()

## Using pygris

In [28]:
zip_codes_list = pygris.zctas()

Using the default year of 2021


In [29]:
zip_codes_list.head()

,ZCTA5CE20,GEOID20,CLASSFP20,MTFCC20,FUNCSTAT20,ALAND20,AWATER20,INTPTLAT20,INTPTLON20,geometry
0,47236,47236,B5,G6350,S,1029067,0,+39.1517426,-085.7252769,"POLYGON ((-85.73410 39.15597, -85.72749 39.156..."
1,47870,47870,B5,G6350,S,8830,0,+39.3701518,-087.4735141,"POLYGON ((-87.47414 39.37016, -87.47409 39.370..."
2,47851,47851,B5,G6350,S,53547,0,+39.5735818,-087.2459626,"POLYGON ((-87.24770 39.57431, -87.24770 39.574..."
3,47337,47337,B5,G6350,S,303089,0,+39.8027537,-085.4372850,"POLYGON ((-85.44357 39.80328, -85.44346 39.803..."
4,47435,47435,B5,G6350,S,13302,0,+39.2657557,-086.2951577,"POLYGON ((-86.29592 39.26547, -86.29592 39.266..."


In [30]:
list_zipcode_2 = zip_codes_list['ZCTA5CE20'].dropna().drop_duplicates().astype(str).tolist()

# add preceding 0 to make 4-digit zip codes into 5-digit
list_zipcode_2 = [zipcode.zfill(5) for zipcode in list_zipcode]

In [31]:
# Convert both lists to sets
set_zipcode = set(list_zipcode)
set_zipcode_2 = set(list_zipcode_2)

# Find the intersection of the two sets
common_zipcodes = set_zipcode.intersection(set_zipcode_2)

# Print the number of common zip codes
print("Number of common zip codes:", len(common_zipcodes))

Number of common zip codes: 25952


In [32]:
len(common_zipcodes)/len(list_zipcode)

1.0

In [71]:
states = pygris.states()
states_list = sorted(states['STUSPS'].unique())

Using the default year of 2021


In [74]:
BG_list = []
for i in states_list:
    BG_list.append(pygris.block_groups(state = i))

Using the default year of 2021
Using FIPS code '02' for input 'AK'
Using the default year of 2021
Using FIPS code '01' for input 'AL'
Using the default year of 2021
Using FIPS code '05' for input 'AR'
Using the default year of 2021
Using FIPS code '60' for input 'AS'
Using the default year of 2021
Using FIPS code '04' for input 'AZ'
Using the default year of 2021
Using FIPS code '06' for input 'CA'
Using the default year of 2021
Using FIPS code '08' for input 'CO'
Using the default year of 2021
Using FIPS code '09' for input 'CT'
Using the default year of 2021
Using FIPS code '11' for input 'DC'
Using the default year of 2021
Using FIPS code '10' for input 'DE'
Using the default year of 2021
Using FIPS code '12' for input 'FL'
Using the default year of 2021
Using FIPS code '13' for input 'GA'
Using the default year of 2021
Using FIPS code '66' for input 'GU'
Using the default year of 2021
Using FIPS code '15' for input 'HI'
Using the default year of 2021
Using FIPS code '19' for input 

In [79]:
BG_df = pd.concat(BG_list, ignore_index=True)

In [86]:
BG_df.head()

,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,GEOID,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,02,090,001300,1,020900013001,Block Group 1,G5030,S,12927348,0,+64.8997414,-147.7944837,"POLYGON ((-147.83013 64.90363, -147.83007 64.9..."
1,02,090,001902,2,020900019022,Block Group 2,G5030,S,132546549,194072,+64.9256133,-147.4350582,"POLYGON ((-147.64405 64.94289, -147.64391 64.9..."
2,02,090,001501,1,020900015011,Block Group 1,G5030,S,35317820,709655,+64.8080966,-147.3165582,"POLYGON ((-147.41165 64.81391, -147.41146 64.8..."
3,02,090,001502,1,020900015021,Block Group 1,G5030,S,7317657,162095,+64.7834800,-147.3245129,"POLYGON ((-147.38760 64.78577, -147.38700 64.7..."
4,02,090,001401,1,020900014011,Block Group 1,G5030,S,9294941,68178,+64.8355818,-147.4563222,"POLYGON ((-147.54345 64.82914, -147.54345 64.8..."


In [82]:
list_censusBG_2 = BG_df['GEOID'].dropna().drop_duplicates().astype(str).tolist()

# add preceding 0 to make 4-digit zip codes into 5-digit
list_censusBG_2 = [censusBG.zfill(12) for censusBG in list_censusBG]

In [88]:
len(list_censusBG_2)

9570

In [83]:
#CensusBG list for NC

list_censusBG = df['censusBlockGroupFips'].dropna().drop_duplicates().tolist()

list_censusBG = [str(int(float(i))) for i in list_censusBG]

# add preceding 0 to make 4-digit zip codes into 5-digit
list_censusBG = [censusBG.zfill(12) for censusBG in list_censusBG]

In [84]:
# Convert both lists to sets
set_bg = set(list_censusBG)
set_bg_2 = set(list_censusBG_2)

# Find the intersection of the two sets
common_bg = set_bg.intersection(set_bg_2)

# Print the number of common zip codes
print("Number of common BG:", len(common_bg))

Number of common BG: 9570


In [85]:
len(common_bg)/len(list_censusBG)

0.08961848931507875

## Using Shapefiles

In [14]:
# Read the shapefile
shapefile_path = 'C:/Users/Asus/Box/Flood Damage PredictionProject/Shapefiles/tl_rd22_us_zcta520/tl_rd22_us_zcta520.shp'  # Replace with the path to your shapefile
map_df = gpd.read_file(shapefile_path)

In [15]:
map_df.head()

,ZCTA5CE20,GEOID20,CLASSFP20,MTFCC20,FUNCSTAT20,ALAND20,AWATER20,INTPTLAT20,INTPTLON20,geometry
0,35592,35592,B5,G6350,S,298552385,235989,+33.7427261,-088.0973903,"POLYGON ((-88.24735 33.65390, -88.24713 33.654..."
1,35616,35616,B5,G6350,S,559506992,41870756,+34.7395036,-088.0193814,"POLYGON ((-88.13997 34.58184, -88.13995 34.582..."
2,35621,35621,B5,G6350,S,117838488,409438,+34.3350314,-086.7270557,"POLYGON ((-86.81659 34.34960, -86.81649 34.349..."
3,35651,35651,B5,G6350,S,104521045,574316,+34.4609087,-087.4801507,"POLYGON ((-87.53087 34.42492, -87.53082 34.429..."
4,36010,36010,B5,G6350,S,335675180,236811,+31.6598950,-085.8128958,"POLYGON ((-85.95712 31.67744, -85.95676 31.677..."


In [21]:
list_zipcode_2 = map_df['ZCTA5CE20'].dropna().drop_duplicates().astype(str).tolist()

# add preceding 0 to make 4-digit zip codes into 5-digit
list_zipcode_2 = [zipcode.zfill(5) for zipcode in list_zipcode]

In [24]:
# Convert both lists to sets
set_zipcode = set(list_zipcode)
set_zipcode_2 = set(list_zipcode_2)

# Find the intersection of the two sets
common_zipcodes = set_zipcode.intersection(set_zipcode_2)

# Print the number of common zip codes
print("Number of common zip codes:", len(common_zipcodes))

Number of common zip codes: 22650


In [26]:
len(common_zipcodes)/len(list_zipcode)

0.8727651048088779